In [1]:
import  time
from    tqdm.notebook           import  tqdm

import  torch
from    torch                   import  Tensor
import  matplotlib.pyplot       as      plt
from    IPython                 import  display
from    ipywidgets              import  Output

import  find_library
from    pytorch.utils                   import  *
from    pytorch.numerical.distribution  import  *
from    pytorch.numerical.solvers       import  *

DTYPE:  torch.dtype     = torch.float64
DEVICE: torch.device    = torch.device('cpu')
tensor_kwargs = {'dtype': DTYPE, 'device': DEVICE}

The following path will be appended.
>>> [/media/junseung/47a90e46-3a9d-467c-bbee-066752b68532/GWANGJAE/python_deep_numerical/pytorch/numerical]
The following path will be appended.
>>> [/media/junseung/47a90e46-3a9d-467c-bbee-066752b68532/GWANGJAE/python_deep_numerical/pytorch]
The following path will be appended.
>>> [/media/junseung/47a90e46-3a9d-467c-bbee-066752b68532/GWANGJAE/python_deep_numerical]


In [2]:
# Time
DELTA_T:    float       = 0.1
MIN_T:      float       = 6.5
MAX_T:      float       = 11.5
NUM_T:      int         = int(1 + (MAX_T - MIN_T + 0.1*DELTA_T) / DELTA_T)
ARR_T:      Tensor      = torch.linspace(MIN_T, MAX_T, NUM_T, **tensor_kwargs)

# Velocity
DIMENSION:      int         = 3
RESOLUTION:     int         = 2**5
MAX_V:          float       = 3.0 / LAMBDA
V:              Tensor      = velocity_grid(
                                    DIMENSION,
                                    RESOLUTION,
                                    MAX_V,
                                    where_closed='left',
                                    **tensor_kwargs
                                )

# Further configurations - FFT
FFT_NORM:   str         = 'forward'
_FREQ:      Tensor      = fft_index(RESOLUTION, **tensor_kwargs)
FREQ_MIN:   int         = int(_FREQ.min())
FREQ_MAX:   int         = int(_FREQ.max())
freqs:      Tensor      = freq_tensor(DIMENSION, RESOLUTION, **tensor_kwargs)
    
# VHS model
VHS_ALPHA:      float   = 0.0
VHS_COEFF:      float   = 1 / area_of_unit_sphere(DIMENSION)

# Metric
METRIC_ORDER:   float   = float('1')

# Problem type
PROBLEM_TYPE:   str = 'bkw' # 'bimodal', 'bkw', 'maxwellian'
PROBLEM_TYPE = PROBLEM_TYPE.lower()

## BKW solution
BKW_COEFF_EXT:  float = get_bkw_coeff_ext(DIMENSION)

## Maxwellian distribution
MAXWELLIAN_MEAN_DENSITY:        float       = torch.ones((1,1), **tensor_kwargs)
MAXWELLIAN_MEAN_VELOCITY:       Tensor      = torch.zeros((1,DIMENSION), **tensor_kwargs)
MAXWELLIAN_MEAN_TEMPERATURE:    float       = torch.ones((1,1), **tensor_kwargs)

## Bimodal distribution
BIMODAL_MODE1_MEAN_DENSITY:     float       = 0.5 * torch.ones((1,1), **tensor_kwargs)
BIMODAL_MODE1_MEAN_VELOCITY:    Tensor      = torch.tensor([[MAX_V/6, *zeros(DIMENSION-1)]], **tensor_kwargs)
BIMODAL_MODE1_MEAN_TEMPERATURE: float       = 0.5 * torch.ones((1,1), **tensor_kwargs)

In [3]:
print(f"[ Initialization ]")
print(f"* Setting the initial value.")
init_f: Tensor
arr_f:  Tensor
if PROBLEM_TYPE == 'bkw':
    bkw_sol = bkw(
        ARR_T, V,
        coeff_ext   = BKW_COEFF_EXT,
        vhs_coeff   = VHS_COEFF,
    )
    init_f = bkw_sol[:, 0]
elif PROBLEM_TYPE == 'maxwellian':
    init_f = maxwellian_homogeneous(
        V,
        MAXWELLIAN_MEAN_DENSITY,
        MAXWELLIAN_MEAN_VELOCITY,
        MAXWELLIAN_MEAN_TEMPERATURE,
    )
elif PROBLEM_TYPE == 'bimodal':
    init_f = maxwellian_homogeneous(
        V,
        BIMODAL_MODE1_MEAN_DENSITY,
        +BIMODAL_MODE1_MEAN_VELOCITY,
        BIMODAL_MODE1_MEAN_TEMPERATURE,
    ) + \
    maxwellian_homogeneous(
        V,
        BIMODAL_MODE1_MEAN_DENSITY,
        -BIMODAL_MODE1_MEAN_VELOCITY,
        BIMODAL_MODE1_MEAN_TEMPERATURE,
    )
arr_f = init_f

print(f"* Computing the kernel components.")
sm_object = FastSM_Boltzmann_VHS(
    dimension   = DIMENSION,
    v_num_grid  = RESOLUTION,
    v_max       = MAX_V,
    vhs_coeff   = VHS_COEFF,
    vhs_alpha   = VHS_ALPHA,
    restitution = 1.0,
    
    dtype   = DTYPE,
    device  = DEVICE,
)
print(f"\tDone. (approximation level: {sm_object.approximation_level})")

fft_config = {'s': sm_object.v_shape, 'dim': sm_object.v_axes, 'norm': FFT_NORM}
arr_f_fft: Tensor = torch.fft.fftn(arr_f, **fft_config)

[ Initialization ]
* Setting the initial value.
----------[ BKW solution ]----------
* coeff_ext: 1.0
* coeff_int: 0.16666666666666663
------------------------------------
* Computing the kernel components.
	Done. (approximation level: 832)


/media/junseung/47a90e46-3a9d-467c-bbee-066752b68532/GWANGJAE/python_deep_numerical/pytorch/numerical/solvers/_kernel_modes/boltzmann_VHS.py:239: Warning: For efficient implementation, the kernel modes for the 3-dimensional case should be referred by computing the 2-norms of the addition and subtraction of two input frequencies. Use the method 'compute_integral_indices' of this class.
  warnings.warn(


In [4]:
# Define the function which returns the target
def return_target(t_idx: int) -> Tensor:
    if PROBLEM_TYPE == 'bkw':
        # Return the BKW solution at the given time index
        return bkw_sol[:, t_idx]
    elif PROBLEM_TYPE == 'maxwellian':
        # Return the initial value, which is the Maxwellian distribution
        return init_f
    elif PROBLEM_TYPE == 'bimodal':
        # Return the Maxwellian distribution
        _batch_size, _data_dim = init_f.size(0), init_f.size(-1)
        _squeeze_dims = tuple(range(1, 1+2*DIMENSION))
        _moments = compute_moments_homogeneous(init_f, V)
        _moments = tuple(map(lambda x: torch.squeeze(x, dim=_squeeze_dims), _moments))
        return maxwellian_homogeneous(V, *_moments).reshape(
            _batch_size, *ones(DIMENSION), *sm_object.v_shape, _data_dim
        )

In [5]:
# Define storages
arr_distribution        = []
arr_mean_density        = []
arr_mean_velocity       = []
arr_mean_temperature    = []
arr_rel_error           = []


# Define the function which updates the storages
def update_storages(t_idx: int, f: Tensor) -> None:
    _mean_density, _mean_velocity, _mean_temperature = \
        compute_moments_inhomogeneous(f, V, eps=1e-20)
    _target = return_target(t_idx)
    _rel_error = relative_error(f, _target, p=METRIC_ORDER)
    
    arr_distribution.append(f)
    arr_mean_density.append(_mean_density)
    arr_mean_velocity.append(_mean_velocity)
    arr_mean_temperature.append(_mean_temperature)
    arr_rel_error.append(_rel_error)
    
    return

In [6]:
print(f"[ Computation ]")
print(f"* Computing the numerical solution.")
_idx_start = 1
out = Output()
display.display(out)

elapsed_time = time.time()
update_storages(0, init_f)
for t_idx, t_curr in enumerate(tqdm(ARR_T[_idx_start:]), _idx_start):
    """Time marching from the temporal index `t_idx-1` to `t_idx`"""
    # Conduct time-marching
    t_prev = float(ARR_T[t_idx-1])
    arr_f_fft = sm_object.forward(t_prev, arr_f_fft, DELTA_T, one_step_RK4_classic)
    arr_f = torch.fft.ifftn(arr_f_fft, **fft_config).real
    # Save data
    update_storages(t_idx, arr_f)
elapsed_time = time.time() - elapsed_time
print(f"\tDone. (Elapsed time: {elapsed_time:.2f} seconds)")

print(f"* Stacking the results into tensors.")
ARR_T                   = ARR_T.cpu()
arr_distribution        = torch.stack(arr_distribution, dim=1).cpu()
arr_mean_density        = torch.stack(arr_mean_density, dim=1).cpu()
arr_mean_velocity       = torch.stack(arr_mean_velocity, dim=1).cpu()
arr_mean_temperature    = torch.stack(arr_mean_temperature, dim=1).cpu()
arr_rel_error           = torch.stack(arr_rel_error, dim=1).cpu()
print(f"\tDone.")

[ Computation ]
* Computing the numerical solution.


Output()

  0%|          | 0/50 [00:00<?, ?it/s]

	Done. (Elapsed time: 83.40 seconds)
* Stacking the results into tensors.
	Done.


In [7]:
_C_times_omega_d = VHS_COEFF*area_of_unit_sphere(DIMENSION)
plt.plot(
    ARR_T, *arr_rel_error,
    'r-', linewidth=1,
    label=rf"$C={(_C_times_omega_d):.2f}/\omega_{DIMENSION}, \alpha={VHS_ALPHA:.1f}$")
plt.suptitle(f"Plot of the relative error ($p={METRIC_ORDER}$)")
plt.title(f"Problem type: {PROBLEM_TYPE}")
plt.xlim(MIN_T, MAX_T)
plt.ylim(1e-12, 1e+1)
plt.grid(True)
plt.yscale('log')
plt.legend()
plt.show()

torch.Size([1, 51, 1, 1, 1, 32, 32, 32, 1])


In [8]:
eps = 1e-32
dv = torch.prod( V[*ones(DIMENSION)] - V[*zeros(DIMENSION)] ).item()
arr_positive = torch.where(arr_distribution>=eps, arr_distribution, eps)
entropy = dv * torch.sum(
    arr_positive*arr_positive.log(),
    dim=tuple(range(2, 2+2*DIMENSION)),
)
print(arr_distribution.shape)

plt.figure(figsize=(7, 5))
plt.title("Entropy")
for _ent in entropy:
    plt.plot(ARR_T.cpu(), _ent.cpu(), linewidth=1)
plt.xlim(MIN_T, MAX_T)
plt.grid(True)
plt.show()

End of file